# Feature Engineering Pipline

## Part1 Package & Helper Fuction

In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

###  Mapping Fuction

In [2]:
def map_yz_to_01(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

### Merging Fuction (tabular data -> single table)

In [3]:
def merging_transactions(tran_data,df_train,df_test,prefix):  
    
    tran_data['purchase_date'] = pd.DatetimeIndex(tran_data['purchase_date']).astype(np.int64) * 1e-9

    # Step_1 Aggregating
    agg_dict = {
        'category_1': ['sum', 'mean'],
        'category_2_1.0': ['mean','sum'],
        'category_2_2.0': ['mean','sum'],
        'category_2_3.0': ['mean','sum'],
        'category_2_4.0': ['mean','sum'],
        'category_2_5.0': ['mean','sum'],
        'category_3_A': ['mean','sum'],
        'category_3_B': ['mean','sum'],
        'category_3_C': ['mean','sum'],
        'merchant_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'city_id': ['nunique'],
        'subsector_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std','nunique'],
        'purchase_date': [np.ptp, 'min', 'max','nunique'],
        'month_lag': ['min', 'max','mean','sum',np.ptp],
        'month_diff1': ['mean','max'],
        'purchase_year':['nunique'],
        'weekend': ['sum', 'mean']

        }
    for col in ['hour','weekofyear','dayofweek']:
        agg_dict[col] = ['nunique']

            
    agg_tran_data = tran_data.groupby(['card_id']).agg(agg_dict)
    agg_tran_data.columns = ['_'.join(col).strip() for col in agg_tran_data.columns.values]
    agg_tran_data.reset_index(inplace=True)
    
    transactions_count = (tran_data.groupby('card_id').size().reset_index(name='transactions_count'))
    agg_tran_data = pd.merge(transactions_count, agg_tran_data, on='card_id', how='left')
    agg_tran_data.columns = [prefix + c if c != 'card_id' else c for c in agg_tran_data.columns]

    # Step_2  Merging
    df_train = pd.merge(df_train, agg_tran_data, on='card_id', how='left')
    df_test = pd.merge(df_test, agg_tran_data, on='card_id', how='left')
    return (df_train, df_test)

******

## Part 2 Features for 'train.csv' & 'test.csv'

In [4]:
df_train = pd.read_csv('train.csv', parse_dates=["first_active_month"])
df_test = pd.read_csv('test.csv', parse_dates=["first_active_month"])

### 'month' & 'year'  from 'first active month' 

In [5]:
df_train["month"] = df_train["first_active_month"].dt.month
df_test["month"] = df_test["first_active_month"].dt.month
df_train["year"] = df_train["first_active_month"].dt.year
df_test["year"] = df_test["first_active_month"].dt.year

### Calculate the elapsed time for each card

In [6]:
df_train['elapsed_time'] = (datetime.date(2018, 2, 1) - df_train['first_active_month'].dt.date).dt.days
df_test['elapsed_time'] = (datetime.date(2018, 2, 1) - df_test['first_active_month'].dt.date).dt.days

### Dummy Variable

In [7]:
df_train = pd.get_dummies(df_train, columns=['feature_1', 'feature_2'])
df_test = pd.get_dummies(df_test, columns=['feature_1', 'feature_2'])

******

## Part 3 Feature Engineering for transactions Data

In [8]:
%%time
df_hist_trans = pd.read_csv('historical_transactions.csv',parse_dates=['purchase_date'])
df_new_trans = pd.read_csv('new_merchant_transactions.csv',parse_dates=['purchase_date'])

CPU times: user 1min 12s, sys: 4.22 s, total: 1min 16s
Wall time: 1min 16s


### Reverse Engineering

In [9]:
df_new_trans['purchase_amount'] = np.round(df_new_trans.purchase_amount / 0.00150265118 + 497.06,2)
df_hist_trans['purchase_amount'] = np.round(df_hist_trans.purchase_amount / 0.00150265118 + 497.06,2)

### Mapping 'authorized_flag', 'category_1' to 0/1

In [10]:
%%time
df_hist_trans = map_yz_to_01(df_hist_trans)
df_new_trans = map_yz_to_01(df_new_trans)

CPU times: user 5.34 s, sys: 652 ms, total: 6 s
Wall time: 5.99 s


### Dummy Variable

In [11]:
%%time
df_hist_trans = pd.get_dummies(df_hist_trans, columns=['category_2', 'category_3'])
df_new_trans = pd.get_dummies(df_new_trans, columns=['category_2', 'category_3'])

CPU times: user 5.78 s, sys: 2.26 s, total: 8.03 s
Wall time: 8.03 s


### Time Features: weekofyear/dayofweek/weekend/hour

In [12]:
%%time
for df in [df_hist_trans,df_new_trans]:
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.dayofweek >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour

CPU times: user 6.95 s, sys: 0 ns, total: 6.95 s
Wall time: 6.95 s


### Time Features: purchase_year & purchase_month

In [13]:
%%time
df_hist_trans['purchase_month'] = df_hist_trans['purchase_date'].dt.month
df_new_trans['purchase_month'] = df_new_trans['purchase_date'].dt.month

df_hist_trans['purchase_year'] = df_hist_trans['purchase_date'].dt.year
df_new_trans['purchase_year'] = df_new_trans['purchase_date'].dt.year

CPU times: user 3.03 s, sys: 4 ms, total: 3.04 s
Wall time: 3.04 s


In [14]:
## Transfer month to continuous variable
(df_hist_trans.loc[df_hist_trans['purchase_year']==2018,'purchase_month']) += 12
(df_new_trans.loc[df_new_trans['purchase_year']==2018,'purchase_month']) += 12

### Time Features: month_diff

In [15]:
%%time
df_hist_trans['month_diff1'] = ((datetime.date(2018, 2, 1) - df_hist_trans['purchase_date'].dt.date).dt.days)//30
df_hist_trans['month_diff1'] += df_hist_trans['month_lag']
df_new_trans['month_diff1'] = ((datetime.date(2018, 2, 1) - df_new_trans['purchase_date'].dt.date).dt.days)//30
df_new_trans['month_diff1'] += df_new_trans['month_lag']

CPU times: user 2min 1s, sys: 904 ms, total: 2min 2s
Wall time: 2min 2s


******

### Separating out unauthorized transactions

In [16]:
df_hist_trans_authorized = df_hist_trans[df_hist_trans['authorized_flag'] == 1]
df_hist_trans_unauthorized = df_hist_trans[df_hist_trans['authorized_flag'] == 0]

In [17]:
for df in [df_hist_trans_authorized,df_hist_trans_unauthorized,df_new_trans]:
    df.loc[df['installments']==-1,'installments'] = 0
    df.loc[df['installments']==999,'installments'] = 0

/home/ubuntu/.local/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Release Memory Usage

In [20]:
del df_hist_trans
gc.collect()

8

---

## Part 4 Extrating Features from Transactions Data

In [21]:
%%time
df_train, df_test = merging_transactions(df_hist_trans_authorized, df_train, df_test,'authorized_')
df_train, df_test = merging_transactions(df_hist_trans_unauthorized, df_train, df_test,'unauthorized_')
df_train, df_test = merging_transactions(df_new_trans, df_train, df_test,'new_')

CPU times: user 8min 48s, sys: 3.53 s, total: 8min 51s
Wall time: 8min 51s


In [29]:
df_train.shape

(201917, 185)

In [30]:
df_test.shape

(123623, 183)

In [27]:
for df in [df_train,df_test]:
    df['hist_total_count'] = df.authorized_transactions_count + df.unauthorized_transactions_count
    df.loc[df['hist_total_count'].isna(),'hist_total_count'] = df.loc[df['hist_total_count'].isna(),'authorized_transactions_count']

### Tagging outliers

In [25]:
df_train['outliers'] = 0
df_train.loc[df_train['target']<-30,'outliers'] = 1 

### Release Memory Usage

In [28]:
del df_hist_trans_authorized ,df_hist_trans_unauthorized ,df_new_trans
gc.collect()

227

### Saving Processed Data

In [31]:
%%time
df_train.to_csv('train_new_clean.csv',index=False)
df_test.to_csv('test_new_clean.csv',index=False)

CPU times: user 58.8 s, sys: 424 ms, total: 59.2 s
Wall time: 59.6 s


### Testing Distribution of Train/Test Set

In [34]:
# ks_2sample test
from scipy.stats import ks_2samp
list_p_value =[]

for i in df_test.columns[3:]:
    list_p_value.append(ks_2samp(df_test[i] , df_train[i])[1])

Se = pd.Series(list_p_value, index = df_test.columns[3:]).sort_values() 
list_discarded = list(Se[Se < .1].index)

In [35]:
list_discarded

['new_purchase_amount_mean',
 'authorized_purchase_amount_sum',
 'unauthorized_category_2_1.0_mean',
 'authorized_category_2_3.0_sum']

In [36]:
Se

new_purchase_amount_mean                     0.043279
authorized_purchase_amount_sum               0.054246
unauthorized_category_2_1.0_mean             0.070209
authorized_category_2_3.0_sum                0.089007
authorized_category_2_3.0_mean               0.100639
new_month_lag_mean                           0.112871
new_purchase_amount_sum                      0.121436
authorized_purchase_amount_min               0.122635
new_category_2_1.0_mean                      0.135876
unauthorized_category_2_1.0_sum              0.139783
authorized_category_2_1.0_mean               0.163847
authorized_purchase_amount_max               0.171231
new_category_2_3.0_mean                      0.177672
authorized_purchase_amount_std               0.181861
new_category_2_3.0_sum                       0.196148
new_category_3_C_mean                        0.200871
new_purchase_month_mean                      0.205667
new_purchase_amount_max                      0.210023
authorized_purchase_month_nu